<a href="https://colab.research.google.com/github/yk-Jeong/nextlab_collabo/blob/main/CP2_%EA%B8%B0%EC%97%85%ED%98%91%EC%97%85_%EB%84%A5%EC%8A%A4%ED%8A%B8%EB%9E%A9(1)_%EB%89%B4%EC%8A%A4%EC%B9%B4%ED%85%8C%EA%B3%A0%EB%A6%AC%EB%B6%84%EB%A5%98.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**1. 뉴스 카테고리 분류**
2. 한글 띄어쓰기 검사
3. 단락 연결 여부 판별

#뉴스 카테고리 분류

9가지 카테고리(culture, economy, entertainment, health, ITscience, life, politic, social, sport)의 다중분류문제 

###라이브러리 설치 및 로드

In [1]:
!pip install mxnet
!pip install gluonnlp tqdm
!pip install sentencepiece
!pip install transformers

     |████████████████████████████████| 47.3 MB 82 kB/s 
  Attempting uninstall: graphviz
    Found existing installation: graphviz 0.10.1
    Uninstalling graphviz-0.10.1:
      Successfully uninstalled graphviz-0.10.1
     |████████████████████████████████| 344 kB 5.3 MB/s 
  Created wheel for gluonnlp: filename=gluonnlp-0.10.0-cp37-cp37m-linux_x86_64.whl size=595720 sha256=81442a9c16c3ee9704bb5727a83dd0dbf690dce746995fe490c3e7d324015037
  Stored in directory: /root/.cache/pip/wheels/be/b4/06/7f3fdfaf707e6b5e98b79c041e023acffbe395d78a527eae00
Successfully built gluonnlp
     |████████████████████████████████| 1.2 MB 5.3 MB/s 
     |████████████████████████████████| 3.8 MB 5.3 MB/s 
     |████████████████████████████████| 6.5 MB 35.6 MB/s 
     |████████████████████████████████| 67 kB 5.2 MB/s 
     |████████████████████████████████| 895 kB 50.7 MB/s 
     |████████████████████████████████| 596 kB 42.6 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
 

In [2]:
!pip install git+https://git@github.com/SKTBrain/KoBERT.git@master

  Cloning https://****@github.com/SKTBrain/KoBERT.git (to revision master) to /tmp/pip-req-build-__byiux4
  Running command git clone -q 'https://****@github.com/SKTBrain/KoBERT.git' /tmp/pip-req-build-__byiux4
     |████████████████████████████████| 132 kB 5.3 MB/s 
     |████████████████████████████████| 4.5 MB 30.0 MB/s 
     |████████████████████████████████| 8.6 MB 31.8 MB/s 
     |████████████████████████████████| 79 kB 4.1 MB/s 
     |████████████████████████████████| 127 kB 31.4 MB/s 
  Created wheel for kobert: filename=kobert-0.2.3-py3-none-any.whl size=15674 sha256=cf0d67fac310fe4ca893a9b3d6c49733f1598da1f2803e13b03807ce283d0855
  Stored in directory: /tmp/pip-ephem-wheel-cache-syma04my/wheels/d3/68/ca/334747dfb038313b49cf71f84832a33372f3470d9ddfd051c0
Successfully built kobert
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
ERROR: pip's dependency resolver does not

In [3]:
import torch 
from torch import nn

import torch.nn.functional as F
import torch.optim as optim

from torch.utils.data import Dataset, DataLoader

import gluonnlp as nlp
import numpy as np
import pandas as pd
import os

from tqdm import tqdm, tqdm_notebook

In [4]:
from kobert.utils import get_tokenizer
from kobert.pytorch_kobert import get_pytorch_kobert_model

from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup

###데이터 로드 

In [5]:
from google.colab import drive 
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
!unzip -uq "/content/drive/My Drive/news_class9x13000.zip" -d "/content/dataset"
!ls 

dataset  drive	sample_data


In [7]:
#아래 라인이 실행되지 않을 경우 숨겨진 파일인 .ipynb_checkpoints를 삭제

!rmdir /content/dataset/2/.ipynb_checkpoints
#참고 https://wooono.tistory.com/252

rmdir: failed to remove '/content/dataset/2/.ipynb_checkpoints': No such file or directory


데이터 확인 및 분리 

In [10]:
#폴더명 변경: dataset 폴더에 있는 하위 카테고리명을 0에서 8까지의 숫자로 변경
#d이미 변경하였다면 미실행

os.listdir('/content/dataset')
os.rename('/content/dataset/culture', '/content/dataset/0')
os.rename('/content/dataset/economy', '/content/dataset/1')
os.rename('/content/dataset/entertainment', '/content/dataset/2')
os.rename('/content/dataset/health', '/content/dataset/3')
os.rename('/content/dataset/ITscience', '/content/dataset/4')
os.rename('/content/dataset/life', '/content/dataset/5')
os.rename('/content/dataset/politic', '/content/dataset/6')
os.rename('/content/dataset/social', '/content/dataset/7')
os.rename('/content/dataset/sport', '/content/dataset/8')

['entertainment',
 'culture',
 'economy',
 'social',
 'sport',
 'ITscience',
 'health',
 'politic',
 'life']

In [8]:
#기사 내용과 카테고리를 하나의 데이터로 이루어진 리스트로 만듦

dataset_train = []
dataset_test = []

root = "/content/dataset/"
list = os.listdir(root)
for cat in list:
    files = os.listdir(root + cat)
    for i,f in enumerate(files):
        fname = root + cat + "/" + f
        file = open(fname, "r", encoding="utf-8")
        strings = file.read()
        if i>=2600: #카테고리당 13,000개의 데이터가 있으며 훈련 세트의 비중을 20%로 설정하였음
            dataset_train.append([strings, cat])
        else:
            dataset_test.append([strings, cat])
        file.close()

print(len(dataset_train), len(dataset_test))

93600 23400


In [9]:
dataset_train[0], dataset_test[0]

(['앙 선생님, 이렇게 사셨는줄 몰랐습니다\n부고기사를 쓰는 일이란 꽤나 묘한 감정을 불러일으킵니다.\n생의 유한함을 일깨우기도 하고, 유명인일 경우 망자에 대한 개인적 추억을 곱씹게도 만들지요.\n오죽했으면 <클로저>의 주인공이자 부고기사 전문 기자 댄은 어떻게 하면 생전 고인의 명예를 훼손시키지 않을까가 최대 고민이었을까요.\n그가 떠난 첫 번째 새벽, 무섭게도 비가 내렸습니다.\n혹자는 하늘도 울었다\n라는 문학적 표현을 쓸지도 모를 일이지요.\n흰색으로 각인된 그가 떠난 첫날밤이 그렇게 가고 있었습니다.\n그의 이름, 앙드레 김.\n유명인들이 떠난 자리에는 대중들의 무수한 평이 남습니다.\n심지어 자살한 유명 여배우에게 악플을 다는 이들도 있었으니 더 말하면 무엇 할까요.\n그러나 의아할 만큼, 패션 디자이너 앙드레 김 선생 부고 소식에는 애도와 상찬, 안타까움의 수사들로 넘쳐나고 있습니다.\n그가 그만큼 사회적인 존경과 유명세, 그리고 호감을 동시에 받았다는 방증이겠지요.\n그런데 문득 인간 앙드레 김에 대해 우리가 무엇을 알고 있나 하는 점에 생각이 미쳤습니다.\n연예인들의 단골 성대모사와 독특한 헤어스타일과 의상, 말투, 그러니까 일종의 캐리커처로만 남은 것은 아닌가 합니다.\n앙드레김 당신은 누구셨습니까?\n하늘에서도 옷을 만들 그의 안타까운 죽음\n한국을 대표하는 패션 디자이너, 민간 문화외교 사절…. 유달리 최초란 수식어가 많이 따라 붙는 패션 디자이너 앙드레 김. 2005년부터 대장암과 싸워오다 최근 폐렴에 의한 합병증으로 유명을 달리한 그의 나이는 향년 75세였습니다.\n따뜻하게 함께 녹화해 주셨던 기억이 있습니다.\n하늘나라 선녀님들의 의상이 더 예뻐지겠네요.\n환한 그 미소로 내내 평안하시길 빕니다.\n삼가 고인의 명복을 빕니다.\n방송인 김제동씨가 트위터에 올린 추모 글입니다.\n이렇게 우리가 기억하는 앙드레 김은 항상 인자한 미소를 지으며 어눌한 듯 화려한 수사로 스포트라이트를 받았던 유명인이었습니다.\n그 중심에 패션이 있었고 

######tokenizer 선언

In [26]:
#device setting 및 사전학습된 bert model load
device = torch.device("cuda:0")
bertmodel, vocab = get_pytorch_kobert_model()

using cached model. /content/.cache/kobert_v1.zip
using cached model. /content/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece


In [11]:
tokenizer = get_tokenizer()
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower = False)

using cached model. /content/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece


######데이터셋 클래스 선언

In [12]:
class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer, max_len, pad, pair):
        transform = nlp.data.BERTSentenceTransform(
            bert_tokenizer, max_seq_length=max_len, pad=pad, pair=pair)

        self.sentences = [transform([i[sent_idx]]) for i in dataset]
        self.labels = [np.int32(i[label_idx]) for i in dataset]

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))

    def __len__(self):
        return (len(self.labels))

######모델 생성

In [13]:
#params

max_len = 64
batch_size = 64
warmup_ratio = 0.1
num_epochs = 10
max_grad_norm = 1
log_interval = 200
learning_rate =  5e-5

In [41]:
data_train = BERTDataset(dataset_train, 0, 1, tok, max_len, True, False)
data_test = BERTDataset(dataset_test, 0, 1, tok, max_len, True, False)

train_dataloader = torch.utils.data.DataLoader(data_train, batch_size=batch_size, num_workers=5, shuffle=True)
test_dataloader = torch.utils.data.DataLoader(data_test, batch_size=batch_size, num_workers=5, shuffle=True)

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


In [42]:
class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes=9,
                 dr_rate=None,
                 params=None):
        super(BERTClassifier, self).__init__()

        self.bert = bert
        self.dr_rate = dr_rate                 
        self.classifier = nn.Linear(hidden_size , num_classes)

        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)
    
    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)
        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device), return_dict=False)
        if self.dr_rate:
            out = self.dropout(pooler)
        return self.classifier(out)

In [43]:
model = BERTClassifier(bertmodel, dr_rate=0.5).to(device)

no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]

optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)
loss_fn = nn.CrossEntropyLoss()

t_total = len(train_dataloader) * num_epochs
warmup_step = int(t_total * warmup_ratio)

scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=warmup_step, num_training_steps=t_total)

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,


In [44]:
#accuracy 계산 
def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc

In [ ]:
for e in range(num_epochs):
    train_acc = 0.0
    test_acc = 0.0
    model.train()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(train_dataloader)):
        optimizer.zero_grad()
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        loss = loss_fn(out, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
        optimizer.step()
        scheduler.step()
        train_acc += calc_accuracy(out, label)
    print("epoch {} train acc {}".format(e+1, train_acc / (batch_id+1)))
    model.eval()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_dataloader)):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        test_acc += calc_accuracy(out, label)
    print("epoch {} test acc {}".format(e+1, test_acc / (batch_id+1)))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  """


  0%|          | 0/1463 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


epoch 1 train acc 0.7386897641831852


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:20: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


...

  0%|          | 0/1463 [00:00<?, ?it/s]

epoch 6 train acc 0.9796116712235133


  0%|          | 0/366 [00:00<?, ?it/s]

epoch 6 test acc 0.8715761612021857


  0%|          | 0/1463 [00:00<?, ?it/s]

testModel: 모델 테스트를 위한 함수 구축 

In [ ]:
def softmax(vals, idx):
    valscpu = vals.cpu().detach().squeeze(0)
    a = 0
    for i in valscpu:
        a += np.exp(i)
    return ((np.exp(valscpu[idx]))/a).item() * 100

def testModel(model, seq): 
  categories = ['culture', 'economy', 'entertainment', 'health', 'ITscience', 'life', 'politic', 'social', 'sport']
  tmp = [seq]
  transform = nlp.data.BERTSentenceTransform(tok, max_len, pad=True, pair=False)
  tokenized = transform(tmp)
  
  model.eval()
  result = model(torch.tensor([tokenized[0]]).to(device), [tokenized[1]], torch.tensor(tokenized[2]).to(device))
  idx = result.argmax().cpu().item()
  print("category:", categories[idx])
  print("accuracy:", "{:.2f}%".format(softmax(result,idx)))

10건의 기사문을 추출해 모델 테스트 

In [ ]:
testModel(model, "겨우 13건 신청…'양육비 미지급자 공개' 유명무실")

#https://www.sedaily.com/NewsView/2623NRMOF3 실제 뉴스는 '사회' 카테고리였음 

In [ ]:
testModel(model, "“여가부 없애려는 한국 보수, 자기개혁 실패 보여주는 결정적 사례")

#https://www.sedaily.com/NewsView/2623NRMOF3 실제 뉴스는 '사회' 카테고리였음 